# 05_Refined_Model_Evaluation
This notebook serves to show the accuracy of the refined model, and how it is better than just one individual setting for the generated features.

### Prerequisites:
- `make refined_model` - if you haven't already run the prerequisites, this will run
    - `make download`
    - `make features`
    - `make extended_features`
    
### Purpose:
The purpose of the refined model is to provide some of the functionality of the extended model (including features calculated with multiple hyperparameter settings and epoch sizes), while not including every setting that is used for the extended model, because calculating that for multiple different edf's is time consuming and memory-intensive. The idea behind the refined model is to look at each sleep state separately—Active Waking, Quiet Waking, Drowsiness, SWS, and REM—and include the single best epoch & welch setting for EEG and ECG for each of them. In this way, you are getting some of the benefit of including the same features calculated at multiple different settings (for example maybe a power spectral density window a.k.a. welch size of 16 seconds is better to predict active waking, but a welch size of 1 second is better for predicting drowsiness), while saving time in only calculating the features 5 times instead of the ~40 different settings used for the extended model.

In [10]:
import pandas as pd
import pytz

In [11]:
import sys
sys.path.insert(0, '..')
import src.models.build_model_LGBM as bmodel
import src.models.build_extended_model_LGBM as emodel

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load features dataframes

In [13]:
# PST timezone
pst_timezone = pytz.timezone('America/Los_Angeles')

# Load features
basic_features_df = pd.read_csv(
    '../data/processed/features/test12_Wednesday_07_features_with_labels.csv', index_col=0
)
refined_features_df = pd.read_csv(
    '../data/processed/features/test12_Wednesday_08_refined_features_with_labels_v3.csv', index_col=0
)


# Set index as DatetimeIndex
basic_features_df.index = pd.DatetimeIndex(basic_features_df.index, tz=pst_timezone)
refined_features_df.index = pd.DatetimeIndex(refined_features_df.index, tz=pst_timezone)

In [16]:
with open ('../reports/results/refined_model_included_settings.txt', 'r') as f:
    custom_settings = [setting.strip('\n') for setting in f.readlines()]

In [17]:
custom_settings

['EPOCH_128_WELCH_1_EEG',
 'EPOCH_128_WELCH_16_EEG',
 'EPOCH_128_WELCH_4_EEG',
 'EPOCH_64_WELCH_4_EEG',
 'EPOCH_32_WELCH_1_EEG',
 'EPOCH_512_WELCH_512_HR',
 'EPOCH_256_WELCH_64_HR',
 'EPOCH_256_WELCH_256_HR',
 'EPOCH_512_WELCH_256_HR',
 'EPOCH_512_WELCH_64_HR']

## Basic model

In [18]:
accs, class_accs, conf_matrs, conf_matr = bmodel.evaluate_model(basic_features_df, 'Simple.Sleep.Code',
                                                                verbosity=0)

Fold 1/5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Fold 2/5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Fold 3/5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Fold 4/5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Fold 5/5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Overall accuracy: 76.43%

Mean class accuracies across folds:
Active Waking    90.47
Drowsiness       40.38
Quiet Waking     52.35
REM              57.86
SWS              81.62
Unscorable        0.00
dtype: float64

Overall confusion matrix:
                      Predicted_Active_Waking    Predicted_Quiet_Waking    Predicted_Drowsiness    Predicted_SWS    Predicted_REM    Predicted_Unscorable
------------------  -------------------------  ------------------------  ----------------------  ---------------  ---------------  ----------------------
True_

In [20]:
conf_matr

,Predicted_Active_Waking,Predicted_Quiet_Waking,Predicted_Drowsiness,Predicted_SWS,Predicted_REM,Predicted_Unscorable
True_Active_Waking,122471,7029,1367,3429,135,0
True_Quiet_Waking,12522,21715,3056,2162,2751,0
True_Drowsiness,3039,5077,12007,1861,348,0
True_SWS,6388,3930,1069,46137,401,0
True_REM,1357,5592,211,1286,22898,0
True_Unscorable,6150,180,22,102,0,0


## Refined model

In [21]:
accs_ref, class_accs_ref, conf_matrs_ref, conf_matr_ref = bmodel.evaluate_model(refined_features_df,
                                                                                'Simple.Sleep.Code',
                                                                                verbosity=0)

Fold 1/5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Fold 2/5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Fold 3/5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Fold 4/5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Fold 5/5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Overall accuracy: 80.06%

Mean class accuracies across folds:
Active Waking    92.79
Drowsiness       47.39
Quiet Waking     56.86
REM              62.60
SWS              84.12
Unscorable        0.00
dtype: float64

Overall confusion matrix:
                      Predicted_Active_Waking    Predicted_Quiet_Waking    Predicted_Drowsiness    Predicted_SWS    Predicted_REM    Predicted_Unscorable
------------------  -------------------------  ------------------------  ----------------------  ---------------  ---------------  ----------------------
True_

In [22]:
conf_matr_ref

,Predicted_Active_Waking,Predicted_Quiet_Waking,Predicted_Drowsiness,Predicted_SWS,Predicted_REM,Predicted_Unscorable
True_Active_Waking,125908,6338,657,1309,219,0
True_Quiet_Waking,11990,23455,3410,1208,2143,0
True_Drowsiness,2090,5096,13804,1277,65,0
True_SWS,4567,3265,1180,48050,863,0
True_REM,1119,4091,51,1383,24700,0
True_Unscorable,6311,114,29,0,0,0
